In [1]:
%load_ext autoreload
%autoreload 2.0
%matplotlib inline

In [2]:
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy as np
import tensorflow as tf

/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-pack

In [3]:
tf.__version__

'1.14.0'

In [4]:
import sys
sys.path.insert(0, '../.')

In [5]:
from data.segmentation_dataset_loader import *
from data.detection_dataset_loader import *
from model import *
from Trainer import *
from evaluation.evaluate import *

In [6]:
from data.postprocessing.nms import *

In [7]:
def prepare_dataset_feed_dict(model, dataset, train_fusion_rgb, train_fusion_fv_lidar, anchor_values, use_nms, is_training=False):
        # camera_tensor, lidar_tensor, fv_velo_tensor, label_tensor, Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = sess.run(dataset)
        data = list(dataset.get_next(batch_size=1))
#         data = list(next(dataset))

#         for i in range(len(data)):
#             data[i] = np.expand_dims(data[i], axis=0)
        camera_tensor, lidar_tensor, label_tensor, label_up_tensor, label_down_tensor, label_left_tensor, label_right_tensor, Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = data
#         print(np.max(camera_tensor))
        d = {model.train_inputs_rgb: camera_tensor,
                model.train_inputs_lidar: lidar_tensor,
                model.y_true: label_tensor,
                model.y_true_up: label_up_tensor,
                model.y_true_down: label_down_tensor,
                model.y_true_left: label_left_tensor,
                model.y_true_right: label_right_tensor,
                model.Tr_velo_to_cam: Tr_velo_to_cam,
                model.R0_rect: R0_rect,
                model.P3: P3,
                model.shift_h: shift_h,
                model.shift_w: shift_w,
                model.anchors: anchor_values,
                model.use_nms: use_nms,
                model.train_fusion_rgb: train_fusion_rgb,
                model.is_training: is_training}
        return d

In [8]:
def sigmoid(x):
    x = x.astype(np.float128)
    x = 1 / (1 + np.exp(-x))
    x = x.astype(np.float32)
    return x

def convert_prediction_into_real_values(label_tensor, truth_value=None,
            anchors=np.array([3.9, 1.6, 1.5]), 
            input_size=(512, 448), output_size=(128, 112), is_label=False, th=0.5):

    ratio = input_size[0] // output_size[0]
    result = []
    ones_index = np.where(sigmoid(label_tensor[:, :, :, -1])>=th)
    if truth_value is not None:
        ones_index = np.where(truth_value[:, :, :, -1]>=th)
#     print(ones_index)
    if len(ones_index) > 0 and len(ones_index[0]) > 0:
        for i in range(0, len(ones_index[0]), 1):
            x = ones_index[0][i]
            y = ones_index[1][i]
            
            out = np.copy(label_tensor[ones_index[0][i], ones_index[1][i], ones_index[2][i], :])
            anchor = np.array([x+0.5, y+0.5, 0.5, anchors[0], anchors[1], anchors[2]])
#             if not is_label:
#               out[:3] = sigmoid(out[:3])
#             if not is_label:
#                 out[:3] = 2 * sigmoid(out[:3]) - 1
            out[:3] = np.tanh(out[:3])*0.5 * anchor[3:6] + anchor[:3]
            
            out[:2] = out[:2] * ratio
            out[2] = out[2] * 40
            
            out[3:6] = np.square(np.maximum(0, out[3:6])) * anchors
            
            k = ones_index[2][i]
            if not is_label:
              out[6] = sigmoid(out[6]) * np.pi/2 - np.pi/4
            if k == 0 and out[6] < 0:
                out[6] = out[6] + np.pi
                
            out[6] = out[6] + k * (np.pi/2)
                        
            result.append(out)
            
    return np.array(result)

In [9]:
import numpy as np
from PIL import Image, ImageDraw
import math
import numpy.matlib as npm

def convert5Pointto8Point(cx_, cy_, w_, h_, a_):

    theta = math.radians(a_)
    bbox = npm.repmat([[cx_], [cy_]], 1, 5) + \
       np.matmul([[math.cos(theta), math.sin(theta)],
                  [-math.sin(theta), math.cos(theta)]],
                 [[-w_ / 2, w_/ 2, w_ / 2, -w_ / 2, w_ / 2 + 8],
                  [-h_ / 2, -h_ / 2, h_ / 2, h_ / 2, 0]])
    # add first point
    x1, y1 = bbox[0][0], bbox[1][0]
    # add second point
    x2, y2 = bbox[0][1], bbox[1][1]
    # add third point
    #x3, y3 = bbox[0][4], bbox[1][4]   
    # add forth point
    x3, y3 = bbox[0][2], bbox[1][2]
    # add fifth point
    x4, y4 = bbox[0][3], bbox[1][3]

    return [x1, y1, x2, y2, x3, y3, x4, y4]

In [10]:
def get_points(converted_points, calib_path, 
                x_range=(0, 71), y_range=(-40, 40), z_range=(-3.0, 1), 
                size=(512, 448, 40), th=0.5):
    all_result = []
    for converted_points_ in converted_points:
        if sigmoid(converted_points_[-1]) >= th:
            result = [0] * 16
            result[0] = 'Car'
            result[1] = -1
            result[2] = -1
            result[3] = -10
            result[8] = converted_points_[5]
            result[9] = converted_points_[4]
            result[10] = converted_points_[3]
            result[14] = converted_points_[6]
            result[15] = sigmoid(converted_points_[-1])

            calib_data = read_calib(calib_path)

            # x_range=(0, 70)
            # y_range=(-40, 40)
            # z_range=(-2.5, 1)

            x_size = (x_range[1] - x_range[0])
            y_size = (y_range[1] - y_range[0])
            z_size = (z_range[1] - z_range[0])

            x_fac = (size[0]-1) / x_size
            y_fac = (size[1]-1) / y_size
            z_fac = (size[2]-1) / z_size

            x, y, z = -((converted_points_[:3] - size) / np.array([x_fac, y_fac, z_fac])) - np.array([0, -1*y_range[0], -1*z_range[0]]) 
            point = np.array([[x, y, z]])
            box3d_pts_3d = point

            pts_3d_ref = project_velo_to_ref(box3d_pts_3d, calib_data['Tr_velo_to_cam'].reshape((3, 4)))
            pts_3d_ref = project_ref_to_rect(pts_3d_ref, calib_data['R0_rect'].reshape((3, 3)))[0]
            for k in range(3):
                result[11 + k] = pts_3d_ref[k]

            imgbbox = ProjectTo2Dbbox(pts_3d_ref, converted_points_[5], converted_points_[4],
                         converted_points_[3], converted_points_[6], calib_data['P2'].reshape((3, 4)))

            result[4:8] = imgbbox
            all_result.append(result)
    return all_result

In [11]:
def read_label2(label_path, calib_path, shift_h, shift_w, x_range=(0, 71), y_range=(-40, 40), z_range=(-3.0, 1), 
                    size=(512, 448, 40), get_actual_dims=False, from_file=True, translate_x=0, translate_y=0, translate_z=0, ang=0, get_neg=False):

    """
    the file format is as follows: 
    type, truncated, occluded, alpha, bbox_left, bbox_top, bbox_right, bbox_bottom,
    dimensions_height, dimensions_width, dimensions_length, location_x, location_y, location_z,
    rotation_y, score) 
    """
    if from_file:
        lines = []
        with open(label_path, 'r') as label_file:
            lines = label_file.readlines()
    else:
        lines = label_path.split('\n')
    # filter car class
    lines = list(map(lambda x: x.split(), lines))
    if len(lines) > 0:
        if get_neg:
            lines = list(filter(lambda x: len(x) > 0 and ( x[0] not in ['Car', 'Van', 'Truck', 'Tram', 'DontCare']), lines))
            if len(lines) > 0:
                lines = lines[:1]
        else:
            lines = list(filter(lambda x: len(x) > 0 and ( x[0] in ['Car', 'Van', 'Truck', 'Tram']), lines))
    
    def get_parameter(index):
        return list(map(lambda x: x[index], lines))
    
    classes = np.array(get_parameter(0))
    dimension_height = np.array(get_parameter(8)).astype(float)
    dimension_width = np.array(get_parameter(9)).astype(float)
    dimension_length = np.array(get_parameter(10)).astype(float)
    # TODO: take shift into consideration - URGENT
    location_x = np.array(get_parameter(11)).astype(float)
    location_y = np.array(get_parameter(12)).astype(float)
    location_z = np.array(get_parameter(13)).astype(float)
    angles = np.array(get_parameter(14)).astype(float)
    
    # print(len(classes))
    calib_data = read_calib(calib_path)

    locations = np.array([[location_x[i], location_y[i], location_z[i]] for i in range(len(classes))])
    # print(locations)
    if len(locations) > 0 and len(locations[0]) > 0:
        locations = project_rect_to_velo(locations, calib_data['R0_rect'].reshape((3, 3)), calib_data['Tr_velo_to_cam'].reshape((3, 4)))
    # print(locations)
    # print(z_range)
    locations = np.array(list(filter(lambda point: (point[0] >= x_range[0]  and point[0] <= x_range[1])
                                    and (point[1] >= y_range[0] and point[1] <= y_range[1])
                                    and (point[2] >= z_range[0] and point[2] <= z_range[1]) , locations)))

    indx = []
    i = 0
    for point in locations:
        if (point[0] >= x_range[0]  and point[0] <= x_range[1])\
            and (point[1] >= y_range[0] and point[1] <= y_range[1])\
            and (point[2] >= z_range[0] and point[2] <= z_range[1]):
            indx.append(i)
        i += 1

    if len(indx) > 0:
        dimension_height = dimension_height[indx]
        dimension_width = dimension_width[indx]
        dimension_length = dimension_length[indx]
        location_x = location_x[indx]
        location_y = location_y[indx]
        location_z = location_z[indx]
        angles = angles[indx]
        classes = classes[indx]

    if len(locations) > 0:
        locations[:, :3] = locations[:, :3] - np.array([translate_x, translate_y, -translate_z])

    # print('.......')
    # print(len(locations))

    points = [project_point_from_camera_coor_to_velo_coor([location_x[i], location_y[i], location_z[i]], 
                                                        [dimension_height[i], dimension_width[i], dimension_length[i]],
                                                        angles[i],
                                                         calib_data)
                for i in range(len(locations))]
    
    x_size = (x_range[1] - x_range[0])
    y_size = (y_range[1] - y_range[0])
    z_size = (z_range[1] - z_range[0])
            
    x_fac = (size[0]-1) / x_size
    y_fac = (size[1]-1) / y_size
    z_fac = (size[2]-1) / z_size
    if get_actual_dims:
        import math
        for i in range(len(points)):
            b = points[i]
            x0 = b[0][0]
            y0 = b[0][1]
            x1 = b[1][0]
            y1 = b[1][1]
            x2 = b[2][0]
            y2 = b[2][1]
            u0 = -(x0) * x_fac + size[0]
            v0 = -(y0 + 40) * y_fac + size[1]
            u1 = -(x1) * x_fac + size[0]
            v1 = -(y1 + 40) * y_fac + size[1]
            u2 = -(x2) * x_fac + size[0]
            v2 = -(y2 + 40) * y_fac + size[1]
            dimension_length[i] = math.sqrt((v1-v2)**2 + (u1-u2)**2)
            dimension_width[i] = math.sqrt((v1-v0)**2 + (u1-u0)**2)
            dimension_height[i] = math.sqrt((-(b[0][2]+(-1*z_range[1]))*z_fac-(-b[4][2]+z_range[1])*z_fac)**2)

      


    x_range = (x_range[0] + translate_x, x_range[1] + translate_x)
    y_range = (y_range[0] + translate_y, y_range[1] + translate_y)
    z_range = (z_range[0] + translate_z, z_range[1] + translate_z)
    output = [[-(locations[i][0] + -1*x_range[0]) * x_fac + size[0], -(locations[i][1] + -1*y_range[0]) * y_fac + size[1], -(locations[i][2] + -1*z_range[0]) * z_fac + size[2], 
                dimension_height[i], dimension_width[i], dimension_length[i], angles[i]] 
                for i in range(len(locations))]
    # import math
    if ang != 0:
        for i in range(len(locations)):
            w = size[0]
            h = size[1]
            output[i][0], output[i][1] = rotate2((w//2, h//2), (output[i][0], output[i][1]), ang / 57.2958)
            output[i][6] = output[i][6] - ang / 57.2958

    output = list(filter(lambda point: 0 <= point[0] < size[0] and 0 <= point[1] < size[1] and 0 <= point[2] < size[2] , output))
    output = np.array(output)

    return points, output, calib_data['Tr_velo_to_cam'], calib_data['R0_rect'], calib_data['P2']




In [12]:
base_path = '../../../Data'

In [13]:
def data_generator(base_path, i, list_camera_paths, list_lidar_paths, list_label_paths, list_calib_paths, 
                   image_size=(370, 1224), lidar_size=(512, 448, 40), anchors=np.array([3.9, 1.6, 1.5]),
                        augment_translate=False, augment_rotate=False):


                camera_path = list_camera_paths[i]
                lidar_path = list_lidar_paths[i]
                label_path = list_label_paths[i]
                calib_path = list_calib_paths[i]
#             for camera_path, lidar_path, label_path, calib_path in zip(list_camera_paths, list_lidar_paths, list_label_paths, list_calib_paths):
                camera_image, shift_h, shift_w = read_camera(camera_path, image_size)
                lidar_image = read_lidar(lidar_path, calib_path, lidar_size)
                velo_front_view = read_pc_fv(calib_path, lidar_path)
                _, label, Tr_velo_to_cam, R0_rect, P3, directions = read_label(label_path, calib_path, shift_h, shift_w)
               
                label = get_target(label, directions, anchors=anchors)
                camera_image = camera_image / 255.
                lidar_image = lidar_image / 255.
                yield(camera_image, lidar_image, label, 
                            np.concatenate([np.array(Tr_velo_to_cam).reshape((3, 4)), np.array([[0, 0, 0, 1]])], axis=0),
                            np.concatenate([np.concatenate([np.array(R0_rect).reshape((3, 3)), np.array([[0], [0], [0]])], axis=1),  np.array([[0, 0, 0, 1]])], axis=0),
                            np.array(P3).reshape((3, 4)), 
                            np.array([shift_h]), 
                            np.array([shift_w])
                            )

In [14]:
def prepare_dataset_feed_dict_2(model, dataset, train_fusion_rgb, train_fusion_fv_lidar, anchor_values, use_nms, is_training=False):
        # camera_tensor, lidar_tensor, fv_velo_tensor, label_tensor, Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = sess.run(dataset)
#         data = list(dataset.get_next(batch_size=1))
        data = list(next(dataset))

        for i in range(len(data)):
            data[i] = np.expand_dims(data[i], axis=0)
#             print(data[i].shape)
        camera_tensor, lidar_tensor, label_tensor , Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = data
#         print(np.max(camera_tensor))
        d = {model.train_inputs_rgb: camera_tensor,
                model.train_inputs_lidar: lidar_tensor,
                model.y_true: label_tensor,
                model.Tr_velo_to_cam: Tr_velo_to_cam,
                model.R0_rect: R0_rect,
                model.P3: P3,
                model.shift_h: shift_h,
                model.shift_w: shift_w,
                model.anchors: anchor_values,
                model.use_nms: use_nms,
                model.train_fusion_rgb: train_fusion_rgb,
                model.is_training: is_training}
        return d

In [30]:
params = {
    'fusion': False,
    'train_reg': False
}
model = Model(graph=None, **params)

/home/injy/mariam_workspace/env/lib/python3.5/site-packages/torch/hub.py:411: UserWarning: TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead
  warnings.warn('TORCH_MODEL_ZOO is deprecated, please use env TORCH_HOME instead')


In [31]:
training=True

In [32]:
list_files = list(map(lambda x: x.split('.')[0], os.listdir(base_path+'/data_object_image_3/training/image_3')))
random.seed(0)
random.shuffle(list_files)

camera_paths = list(map(lambda x: base_path+'/data_object_image_3/training/image_3/' + x + '.png', list_files))
lidar_paths = list(map(lambda x: base_path+'/data_object_velodyne/training/velodyne/' + x + '.bin', list_files))
label_paths = list(map(lambda x: base_path + '/data_object_label_2/training/label_2/' + x + '.txt', list_files))
calib_paths = list(map(lambda x: base_path + '/data_object_calib/training/calib/' + x + '.txt', list_files))
        
ln = int(len(list_files) * 0.5)
final_sample = len(list_files)
        
if training:
            list_files = list_files[:ln]
            list_camera_paths = camera_paths[:ln]
            list_lidar_paths = lidar_paths[:ln]
            list_label_paths = label_paths[:ln]
            list_calib_paths = calib_paths[:ln]
else:
            list_camera_paths = camera_paths[ln:final_sample]
            list_lidar_paths = lidar_paths[ln:final_sample]
            list_label_paths = label_paths[ln:final_sample]
            list_calib_paths = calib_paths[ln:final_sample]
            list_files = list_files[ln:final_sample]


In [33]:
list_files2 = list(map(lambda x: x.split('.')[0], os.listdir(base_path+'/data_object_image_3/training/image_3')))
random.seed(0)
random.shuffle(list_files2)
ln = int(len(list_files2) * 0.5)
list_files2= list_files2[:ln]

In [34]:
def prepare_dataset_feed_dict2(dataset, train_fusion_rgb, train_fusion_fv_lidar, use_nms, anchors, is_training=True, batch_size=1):

        data = dataset.get_next(batch_size=batch_size)

        # for i in range(len(data)):
        #     data[i] = np.expand_dims(data[i], axis=0)
        if model.params['use_fv']:
            camera_tensor, lidar_tensor, label_tensor, Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = data
            d = {model.train_inputs_rgb: camera_tensor,
                    model.train_inputs_lidar: lidar_tensor,
                    model.y_true: label_tensor,
                    model.Tr_velo_to_cam: Tr_velo_to_cam,
                    model.R0_rect: R0_rect,
                    model.P3: P3,
                    model.shift_h: shift_h,
                    model.shift_w: shift_w,
                    model.use_nms: use_nms,
                    model.train_fusion_rgb: train_fusion_rgb,
                    model.train_fusion_fv_lidar: train_fusion_fv_lidar,
                    model.is_training: is_training,
                    model.anchors: anchors}
        else:
            camera_tensor, lidar_tensor, label_tensor, Tr_velo_to_cam, R0_rect, P3, shift_h, shift_w = data
            d = {model.train_inputs_rgb: camera_tensor,
                    model.train_inputs_lidar: lidar_tensor,
                    model.y_true: label_tensor,                   
                    model.Tr_velo_to_cam: Tr_velo_to_cam,
                    model.R0_rect: R0_rect,
                    model.P3: P3,
                    model.shift_h: shift_h,
                    model.shift_w: shift_w,
                    model.use_nms: use_nms,
                    model.train_fusion_rgb: train_fusion_rgb,
                    model.is_training: is_training,
                    model.anchors: anchors}
        return d

In [36]:
ln * 6

22440

In [63]:
with model.graph.as_default():
            
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True

  with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
#     model.saver.restore(sess, tf.train.latest_checkpoint('../training_files/tmp/'))
#     sess.run(tf.global_variables_initializer())
    anchor_values = prepare_anchors()
#     print(anchor_values.shape)
    anchor_values = np.repeat(anchor_values, 1, axis=0)
#     dataset = DetectionDatasetLoader(base_path='../../../Data', batch_size=1, random_seed=0, training=False)
#     dataset = data_generator(base_path, i_c, list_camera_paths, list_lidar_paths, list_label_paths, list_calib_paths)
    dataset = DetectionDatasetLoader(base_path, None, 0.5, 0, training)
    i = 0
    cls_losses = []
    loc_losses = []
    dim_losses = []
    theta_losses = []
    accs = []
    ious = []
    ious_loc = []
    ious_dim = []
    cls_weights = []
    possible_weights = list(range(500, 100000, 1000))
    while i <= ln:
      cls_weight = i//200
      feed_dict = prepare_dataset_feed_dict2(dataset, 
                                             False, 
                                             False,
                                             False,
                                             anchor_values,
                                             1)
        
      feed_dict[model.learning_rate_placeholder] = 0.0001
      feed_dict[model.cls_weight] = cls_weight

      _, cls_loss, loc_loss, dim_loss, theta_loss,\
            iou, iou_dim, iou_loc, acc = sess.run([model.train_op_lidar,\
                                                              model.classification_loss, model.loc_reg_loss, model.dim_reg_loss, model.theta_reg_loss,\
                                                              model.iou, model.iou_dim, model.iou_loc, model.accuracy_detection],\
                                                             feed_dict=feed_dict)   
      accs.append(acc)
      cls_losses.append(cls_loss)
      cls_weights.append(cls_weight)
      dim_losses.append(dim_loss)
      theta_losses.append(theta_loss)
      loc_losses.append(loc_loss)
      ious.append(iou)
      ious_loc.append(iou_loc)
      ious_dim.append(iou_dim)
        
      if i % 100 == 0:
        print('i = ', i, ", accs = ", np.mean(np.array(accs)))
        
      i += 1
#       break 
    

i =  0 , accs =  0.33333334


KeyboardInterrupt: 

In [65]:
accs

[0.33333334,
 0.33333334,
 0.33333334,
 0.0,
 0.33333334,
 0.33333334,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.7777778,
 0.6666667,
 0.6666667,
 0.5555556,
 0.5555556,
 0.44444445,
 0.5,
 0.5,
 0.71428573,
 0.71428573,
 0.5714286,
 0.42857143,
 0.5,
 0.75,
 0.5,
 0.75,
 0.75,
 0.75,
 0.6666667,
 0.0,
 0.6666667,
 0.6666667,
 0.6666667,
 0.33333334,
 0.0,
 0.16666667,
 0.16666667,
 0.0,
 0.33333334,
 0.0,
 1.0,
 1.0,
 1.0,
 0.6666667,
 1.0,
 1.0,
 0.0,
 0.4]

In [66]:
cls_losses

[0.20533465,
 0.2017749,
 0.20452043,
 0.20049365,
 0.20965265,
 0.20480683,
 0.21735638,
 0.22050263,
 0.21538119,
 0.21982405,
 0.22025508,
 0.21846627,
 0.18973373,
 0.18835539,
 0.19074968,
 0.19117829,
 0.18721981,
 0.19249827,
 0.21393403,
 0.21365494,
 0.20875868,
 0.21145633,
 0.20886998,
 0.2096351,
 0.2099193,
 0.2099193,
 0.20944002,
 0.21275036,
 0.21080366,
 0.21430033,
 0.21759468,
 0.21992403,
 0.22163707,
 0.2186225,
 0.2214932,
 0.22154297,
 0.21634531,
 0.22289029,
 0.21732798,
 0.21313901,
 0.21482411,
 0.21356876,
 0.20295525,
 0.20411004,
 0.204038,
 0.20983666,
 0.21191235,
 0.21115775,
 0.19280943,
 0.19199064,
 0.1933833,
 0.18980268,
 0.19330953,
 0.19239916,
 0.21141961,
 0.20897448]